# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In the next code cell the data is read from Wikipedia. 

In [2]:
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The not assigned boroughs are drop. 

In [3]:
toronto_df.drop(
    toronto_df.loc[toronto_df['Borough'] == 'Not assigned'].index, 
    inplace=True
)
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


The not assigned neighborhoods are renamed like their borough. 

In [4]:
for borough, neighborhood in zip(toronto_df['Borough'], toronto_df['Neighborhood']):
    if neighborhood == 'Not assigned':
        toronto_df['Neighborhood'] = toronto_df['Neighborhood'].replace(neighborhood, borough)
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


The neighborhoods of the same postcode are combined.

In [5]:
toronto_df = toronto_df.groupby('Postcode')['Borough', 'Neighborhood'].agg(', '.join).reset_index()
for i in range(toronto_df.shape[0]):
    toronto_df.loc[i, 'Borough'] = toronto_df.loc[i, 'Borough'].rsplit(',')[0]
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The number of rows of the dataframe is printed.

In [6]:
toronto_df.shape[0]

103